# 文件压缩和解压
[20210622 created,另外设置为独立的一章, 20210622 last updated]

为方便传输,拷贝和保密的需要,常常需要将零散的小文件打包成.zip格式的压缩文件.    
在Python中,可以利用第三方库 zipfile模块中的函数来实现压缩其他类型的文件为zip格式或解压zip文件.     
参考资料:   
[1]zipfile官方中文文档 https://docs.python.org/zh-cn/3/library/zipfile.html
[2]<繁琐工作自动化>
[3]DataWhale编写的<Python办公自动化>教程task01. https://github.com/datawhalechina/team-learning-program/tree/master/OfficeAutomation

#### 创建和添加到zip文件
将上述章节中animals文件夹进行压缩。创建一个example.zip的zip文件，并向其中添加文件。

zipfile.ZipFile('filename.zip', 'w') ：以写模式创建一个压缩文件

ZipFile 对象的 write('filename','compress_type=zipfile.ZIP_DEFLATED')方法：如果向write()方法中传入一个路径，Python 就会压缩该路径所指的文件， 将它加到 ZIP 文件中。 如果向write()方法中传入一个字符串，代表要添加的文件名。第二个参数是“压缩类型”参数，告诉计算机用怎样的算法来压缩文件。可以总是将这个值设置为 zipfile.ZIP_DEFLATED（这指定了 deflate 压缩算法，它对各种类型的数据都很有效）。

注意：写模式会擦除zip文件中所有原有的内容。如果只希望将文件添加到原有的zip文件中，就要向zipfile.ZipFile()传入'a'作为第二个参数，以添加模式打开 ZIP 文件。


In [1]:
## 1 创建一个new.zip压缩文件，并向其中添加一个文件
import zipfile

In [2]:
newZip = zipfile.ZipFile('new.zip','w')
newZip.write('readme.md',compress_type=zipfile.ZIP_DEFLATED)
newZip.close()

In [ ]:
newZip = zipfile.ZipFile('new.zip','w')
newZip.write('D:\\animals\\dogs\\Taidi.txt',compress_type=zipfile.ZIP_DEFLATED)
newZip.close()

In [ ]:
## 2 创建一个example.zip的压缩文件，将animals文件夹下所有文件进行压缩。
import os
newZip = zipfile.ZipFile('example.zip','w')
for folderName, subFolders,fileNames in os.walk('D:\\animals'):
    for filename in fileNames:
        newZip.write(os.path.join(folderName,filename),compress_type=zipfile.ZIP_DEFLATED)
newZip.close()

#### 压缩时加密*
通过即时通讯软件或者邮件等方式传输文件时,为了避免文件泄露,可以将文件压缩时添加密码,并将密码以另外的途径告知对方.       
zipfile库支持解密 ZIP 归档中的加密文件, 但是目前不能创建一个加密的文件,因此需要加密压缩时需要通过其他的方式.   

以下我们选择使用DOS方式的WINRAR来加密压缩.  
参考资料:   
https://blog.csdn.net/weixin_40950781/article/details/105107205

In [1]:
import zipfile
_zipfile = zipfile.ZipFile('new1.zip', 'w', zipfile.ZIP_DEFLATED)
_zipfile.write('readme.md')
_zipfile.setpassword(bytes('20210622','utf-8')) # python3中传入byte后能执行成功.参考https://www.cnblogs.com/xiaoxiao-niao/p/8266768.html
_zipfile.close()
# 完成加密压缩后,测试打开,发现并不需要密码.

In [3]:
import os
import hashlib
import time
from shutil import copyfile

In [ ]:

def getfielpath(path, sub):
    if os.path.isdir(path):
        # 如果绝对路径下的文件夹
        for i in os.listdir(path):  # i文件名
            path2 = os.path.join(path, i)  # 拼接绝对路径
            if os.path.isdir(path2):  # 判断如果是文件夹,调用本身
                getfielpath(path2, sub)
            else:
                sub.append(path2)
    elif os.path.isfile(path):
        # 如果绝对路径下的文件
        sub.append(path)
    else:
        print("File or path doesn\'t exit")


def zipFile(filepath, WinRARpath, password, apt=None):
    oldname = filepath.split('\\')[-1]
    """获取文件的hash"""
    datas = {}  # 结果存为json，以便后需
    f = open(filepath, "rb")
    rb = f.read()
    data = {'MD5': hashlib.md5(rb).hexdigest(),
            'SHA1': hashlib.sha1(rb).hexdigest(),
            'SHA256': hashlib.sha256(rb).hexdigest()}
    f.close()
    """文件hash由文件名构成词典"""
    datas[oldname] = data

    """以文件的sha256重命名：若不重命名则当文件名中有空格或特殊符号时，启动DOS命令会失败"""
    newfilename = filepath.replace(oldname, data['SHA256'])
    """如果使用sha256命名的文件已存在，说明两个文件的sha256值相同，即文件重复，删除文件；否则重命名"""
    if os.path.exists(newfilename):
        os.remove(filepath)
        return 1
    else:
        os.rename(filepath, newfilename)
        filepath = newfilename


    """输出压缩文件的位置，即将压缩文件输出到哪个文件夹中，压缩文件以文件的 SHA256.rar 命名"""
    if os.path.exists(r'D:\zipsampletemp'):
        pass
    else:
        os.makedirs(r'D:\zipsampletemp')

    outputpath = f"D:\\zipsampletemp\\{data['SHA256']}"

    """如果压缩文件已存在，则说明录入重复，删除源文件即可"""
    if os.path.exists(outputpath + ".rar"):
        print(f"[-]--->压缩文件已存在:\t{filepath}")
        os.remove(filepath)
        return 1

    """DOS命令"""
    cmdzip = f"{WinRARpath} a -ep -p{password}  {outputpath} {filepath}"

    try:
        # DOS调用WinRAR加密压缩文件
        os.popen(cmdzip)
    except Exception as err:
        print(err)

    with open("d:\\newsample.txt", "a", encoding="utf-8")as f:
        if apt:
            hashs = data['MD5'] + "#" + data['SHA1'] + "#" + data['SHA256'] + "#" + apt + "\n"
        else:
            hashs = data['MD5'] + "#" + data['SHA1'] + "#" + data['SHA256'] + "#" + "\n"
        f.writelines(hashs)


def unzipFile(filepath, WinRARpath, password, flag):
    outputpath = "D:\\TEMPTEMP"
    cmdunzip = f"{WinRARpath} e -p{password}  {filepath} {outputpath}"
    try:
        # DOS调用WinRAR加密压缩文件
        os.popen(cmdunzip)
        print(f"[+]==>源文件解压成功：\t{filepath}")
        if flag == 0:
            # 删除原有文件
            os.remove(filepath)
            print(f"[+]==>源文件删除成功：\t{filepath}")
        elif flag == 1:
            pass
    except Exception as err:
        print(err)


if __name__ == '__main__':
    WinRARpath = r"D:\WinRAR\Rar.exe"
    password = "这里是设置的默认压缩密码"
    path = r"C:\Users\WIN10\Desktop\samples\112\112\Teamspy(45个)\1578467312"
    # apt = 'unknown'
    apt = 'TeamSpy Crew'

    sub = []
    getfielpath(path, sub)
    for i in sub:
        zipFile(i, WinRARpath=WinRARpath, password=password, apt=apt)
    print(f"[+]===>共计压缩文件个数：\t{len(sub)}")

    # for i in sub:
    #     unzipFile(i, WinRARpath=WinRARpath, password=password, flag = flag)

————————————————
版权声明：本文为CSDN博主「litchi125」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/weixin_40950781/article/details/105107205

#### 读取zip文件
调用zipfile.ZipFile(filename)函数创建一个ZipFile对象（注意大写字母Z和F）,filename是要读取zip文件的文件名。

ZipFile对象中的两个常用方法：

namelis()方法，返回zip文件中包含的所有文件和文件夹的字符串列表。

getinfo()方法，返回一个关于特定文件的ZipInfo对象。

ZipInfo对象的两个属性：file_size和compress_size，分别表示原来文件大小和压缩后文件大小. 

In [ ]:
import zipfile,os
exampleZip = zipfile.ZipFile('example.zip')
exampleZip.namelist()
catInfo = exampleZip.getinfo('animals/Miki.txt')
catInfo.file_size
catInfo.compress_size
print('Compressed file is %s x smaller!' %(round(catInfo.file_size/catInfo.compress_size,2)))
exampleZip.close()

#### 从zip文件中解压缩
ZipFile 对象的 extractall()方法：从zip文件中解压缩所有文件和文件夹，放到当前工作目录中。也可以向extractall()传递的一个文件夹名称，它将文件解压缩到那个文件夹， 而不是当前工作目录。如果传递的文件夹名称不存在，就会被创建。

ZipFile 对象的 extract()方法:从zip文件中解压单个文件。也可以向 extract()传递第二个参数， 将文件解压缩到指定的文件夹， 而不是当前工作目录。如果第二个参数指定的文件夹不存在， Python 就会创建它。extract()的返回值是被压缩后文件的绝对路径。


In [ ]:
import zipfile, os
exampleZip = zipfile.ZipFile('example.zip')
exampleZip.extractall('.\zip')
exampleZip.close()
exampleZip = zipfile.ZipFile('example.zip')
exampleZip.extract('animals/Miki.txt')
exampleZip.extract('animals/Miki.txt', 'D:\\animals\\folders')
exampleZip.close()